# I.M.A.G.E.R.: A Gravitational Microlensing Visualization Tool
I.M.A.G.E.R. (Images of Microlensing And Gravitational Einstein Rings), is to a tool that allow students and educators to visually appreciate the phenomenon of gravitaional lensing. There are two sections, the first is the most simple case where there is no exoplanet present in the lensing system while the second section includes the presence of a singular exoplanet. 

This notebook allows you to change the parameters that affect a graviationally lensed event and see how it effects the location and magnification of the images from the perspective of the observer. This will aid in the intuition behind the mathematical equations presented in class.

Make sure that you have the most updated version of ipywidgets. If you do not have ipywidgets simply use 'pip install ipywidgets' or 'conda install ipywidgets'.

In [1]:
#Run this cell to import all the required packages
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, Label
from IPython.display import display

## Gravitational Lensing with No Exoplanet
Run the following cell once and the interactive sliders will appear along with the plots of the event and magnifcation. These plots will automatically update as you change the value of the slider. If the sliders do not appear please follow the solution to this issue here: https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show. After following those steps, refresh your notebook and restart the kernel; the slider should appear then.

The red dot is the source object, the green dot is the lensing object, and the black and blue dots are the lensed images of the source object.

In [11]:
#Define Physical Constants
c = 3e8
G = 6.67430e-11

#General stylistic and range of time
style = {'description_width': 'initial'}
t = np.arange(-100,100,1)

def update_plot(dist_l, dist_s, mass, v_rel, u, t_0):
    dist_r = (dist_s*3.086e+19 - dist_l*3.086e+19)/(dist_s*3.086e+19*dist_l*3.086e+19) #units of meters [m]
    theta_e = np.sqrt(((4*mass*1.989e+30*G)/(c**2)*dist_r))*206265*1e3 #units of milli arcseconds [mas]
    print('Einstein Radius [mas]:' + str(theta_e))

    crossing_time = dist_l*3.086e+19 * (theta_e/1e3/206265) / (v_rel*3600*24) #units of days
    print('Crossing Time [days]:' + str(crossing_time))

    beta = u*theta_e
    imag_pos = beta + np.sqrt(beta**2+4*theta_e**2) 
    #print('Positive Image:' + str(imag_pos))
    imag_neg = beta - np.sqrt(beta**2+4*theta_e**2)
    #print('Negative Image:' + str(imag_neg))
    dist_ls = dist_s-dist_l

    impact_t = np.sqrt((u)**2 + ((t-t_0)/crossing_time)**2)
    mag = (impact_t**2+2)/impact_t/np.sqrt(impact_t**2+4)

    fig, ax = plt.subplots(ncols=2,figsize=(12,6))
    ax[0].set(xlim=(-10, 10), ylim = (-10, 10))
    ax[0].plot(0,0,'ro')
    ax[0].plot(u,'go')
    ax[0].plot(imag_pos,imag_pos, 'ko')
    ax[0].plot(imag_neg,imag_neg, 'bo')
    ax[0].axhline(u)
    ax[0].set_title('Gravitational Lensing',fontsize='x-large')
    ax[0].set_xlabel(r'$\theta_y [mas]$',fontsize='x-large')
    ax[0].set_ylabel(r'$\theta_x [mas]$',fontsize='x-large')
    ax[0].grid(linestyle = '--')
    a_circle = plt.Circle((0,0), theta_e, fill=False, color = 'k')
    ax[0].add_artist(a_circle)

    ax[1].set(xlim=(-16, 16), ylim = (0, 11))
    ax[1].plot(t,mag)
    ax[1].set_title('Event Light Curve',fontsize='x-large')
    ax[1].set_xlabel('Time t [days]',fontsize='x-large')
    ax[1].set_ylabel('Magnification',fontsize='x-large')
    ax[1].grid(linestyle = '--')
    plt.tight_layout()
    plt.show()

#The interactive slider settings can be updated here
dist_l = widgets.FloatSlider(value=1, min=1e-100, max=8.6, description=r'$D_L:$ [kpc] ', style = style) # units of kpc
dist_s = widgets.FloatSlider(value=2, min=1e-100, max=8.6, description=r'$D_S:$ [kpc]',style = style) 
mass = widgets.FloatSlider(value=1, min=1e-100, max=15.1, description='Lens Mass' + r'$M_{*}[M_{\odot}]$' + ':',style = style) #units of kpc
v_rel = widgets.FloatLogSlider(value = 1 , min=6, max = 10, description = r'$v_{\bot}$ [m/s]:',style = style) #units of m/s
u = widgets.FloatSlider(value = 0.1 , min=-2.1, max = 2.05, description = 'Impact Parameter'+' ' + r'$u_0~[\theta_E]$:',style = style) #units of theta_e
t_0 = widgets.FloatSlider(value = 0 , min=-16, max = 16, description = 'Time of Peak'+' '+ r'$t_0$ [days]:',style = style) #units of days

#This function updates the minimum value of dist_s to be the value of dist_l as there would be no event if the source was closer than the lens
def update_dist_s(*args):
    dist_s.min = dist_l.value
dist_l.observe(update_dist_s, 'value')

#This line performs the live update of the plot and slider widgets as the user interacts with them
widgets.interactive(update_plot, dist_l=dist_l, dist_s=dist_s, mass=mass, v_rel = v_rel, u=u, t_0=t_0)

interactive(children=(FloatSlider(value=1.0, description='$D_L:$ [kpc] ', max=8.6, min=1e-100, style=SliderSty…

## Exoplanet Present in Lensing System
The following cell provides a visual of what it is like if there was an exoplanet present in the system of the lensing object. Notice the additional peaks in the light curve from the lensing of the exoplanet in addition to the lensing object. 

The red dot is the source object, the green dot is the lensing object, the yellow dot is the exoplanet in the system of the lensing object, and the black and blue dots are the lensed images of the source object.

In [10]:
#Define Physical Constants
c = 3e8
G = 6.67430e-11

#General stylistic and range of time
style = {'description_width': 'initial'}
t = np.arange(-100,100,1)

def update_plot(dist_l, dist_s, mass, mass_planet,v_rel, u, t_0):
    dist_r = (dist_s*3.086e+19 - dist_l*3.086e+19)/(dist_s*3.086e+19*dist_l*3.086e+19) #units of meters [m]
    theta_e = np.sqrt(((4*mass*1.989e+30*G)/(c**2)*dist_r))*206265*1e3 #units of milli arcseconds [mas]
    print('Einstein Radius [mas]:' + str(theta_e))

    crossing_time = dist_l*3.086e+19 * (theta_e/1e3/206265) / (v_rel*3600*24) #units of days
    print('Crossing Time [days]:' + str(crossing_time))

    beta = u*theta_e
    imag_pos = beta + np.sqrt(beta**2+4*theta_e**2) 
    #print('Positive Image:' + str(imag_pos))
    imag_neg = beta - np.sqrt(beta**2+4*theta_e**2)
    #print('Negative Image:' + str(imag_neg))
    dist_ls = dist_s-dist_l

    impact_t = np.sqrt((u)**2 + ((t-t_0)/crossing_time)**2)
    q = mass_planet/mass
    new_mag = (impact_t**2+2)/impact_t/np.sqrt(impact_t**2+4)
    i1 = 100 + int(t_0)
    i2 = 99 + int(t_0)
    i3 = 95 + int(t_0)
    i4 = 98 + int(t_0)
    if v_rel > 1e6:
        new_mag[i1]=new_mag[i1]+new_mag[i1]*q
        new_mag[i2]=new_mag[i2]+new_mag[i2]*q
    else:
        new_mag[i3]=new_mag[i3]+new_mag[i3]*q
        new_mag[i4]=new_mag[i4]+new_mag[i4]*q
    
    fig, ax = plt.subplots(ncols=2,figsize=(12,6))
    ax[0].set(xlim=(-10, 10), ylim = (-10, 10))
    ax[0].plot(0,0,'ro')
    ax[0].plot(u,'go')
    ax[0].plot(-.5,u,'yo')
    ax[0].plot(imag_pos,imag_pos, 'ko')
    ax[0].plot(imag_neg,imag_neg, 'bo')
    ax[0].axhline(u)
    ax[0].set_title('Gravitational Microlensing of Planet',fontsize='x-large')
    ax[0].set_xlabel(r'$\theta_y [mas]$',fontsize='x-large')
    ax[0].set_ylabel(r'$\theta_x [mas]$',fontsize='x-large')
    ax[0].grid(linestyle = '--')
    a_circle = plt.Circle((0,0), theta_e, fill=False, color = 'k')
    ax[0].add_artist(a_circle)

    #ax[1].set(xlim=(-16, 16), ylim = (0, 11))
    ax[1].plot(t,new_mag)
    ax[1].set_title('Event Light Curve',fontsize='x-large')
    ax[1].set_xlabel('Time t [days]',fontsize='x-large')
    ax[1].set_ylabel('Magnification',fontsize='x-large')
    ax[1].grid(linestyle = '--')
    plt.tight_layout()
    plt.show()

#The interactive slider settings can be updated here
dist_l = widgets.FloatSlider(value=1, min=1, max=8.6, description=r'$D_L:$ [kpc] ', style = style) # units of kpc
dist_s = widgets.FloatSlider(value=2, min=1, max=8.6, description=r'$D_S:$ [kpc]',style = style) #units of kpc
mass = widgets.FloatSlider(value=2, min=1, max=15.1, description='Lens Mass' + r'$M_{*}[M_{\odot}]$' + ':',style = style)
mass_planet = widgets.FloatSlider(value=1, min=1, max=10.1, description='Exoplanet Mass' + r'$ M_p [M_{\odot}]$' + ':',style = style)
v_rel = widgets.FloatLogSlider(value = 1 , min=6, max = 10, description = r'$v_{\bot}$ [m/s]:',style = style) #units of m/s
u = widgets.FloatSlider(value = 0.1 , min=-2.1, max = 2.05, description = 'Impact Parameter'+' ' + r'$u_0~[\theta_E]$:',style = style) #units of theta_e
t_0 = widgets.FloatSlider(value = 0 , min=-16, max = 16, description = 'Time of Peak'+' '+ r'$t_0$ [days]:',style = style) #units of days

#This function updates the minimum value of dist_s to be the value of dist_l as there would be no event if the source was closer than the lens
def update_dist_s(*args):
    dist_s.min = dist_l.value
dist_l.observe(update_dist_s, 'value')

def update_mass_planet(*args):
    mass_planet.max = mass.value
mass.observe(update_mass_planet, 'value')

#This line performs the live update of the plot and slider widgets as the user interacts with them
widgets.interactive(update_plot, dist_l=dist_l, dist_s=dist_s, mass=mass, mass_planet=mass_planet,v_rel = v_rel, u=u, t_0=t_0)

interactive(children=(FloatSlider(value=1.0, description='$D_L:$ [kpc] ', max=8.6, min=1.0, style=SliderStyle(…